# Title
[]()

In [ ]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [ ]:
with open("credentials.json") as f:
    credentials = json.load(f)

# ig_user_id = credentials['am_ig_user_id']
# access_token = credentials['am_ig_access_token']
ig_user_id = credentials['ig_user_id']
access_token = credentials['access_token']

group_id = credentials['STEMM_group_id']

# Session 1
## Prompt 1 [repeat last prompt from previous session]
Modify the `get_facebook_posts` function to also return a DataFrame that has the each post comment, but without increasing the number of API calls. Each row should have the post_id, comment message, comment time stamp, and comment author. Include the code for calling the function.

Here is the `get_facebook_posts` function to modify:
```python
def get_facebook_posts(group_id, access_token, limit=100):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        pandas.DataFrame: A DataFrame containing the posts and their associated data.
    """
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(0).summary(true),reactions.limit(0).summary(true).type(LIKE).as(like),reactions.limit(0).summary(true).type(LOVE).as(love),reactions.limit(0).summary(true).type(WOW).as(wow),reactions.limit(0).summary(true).type(HAHA).as(haha),reactions.limit(0).summary(true).type(SAD).as(sad),reactions.limit(0).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    data = response.json()
    posts = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })
    df = pd.DataFrame(posts)
    # Reorder columns
    df = df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow_count", "haha_count", "sad_count", "angry_count", "permalink_url"]]
    print("Response Status Code:", status_code)
    return df, data
```
### Response
To modify the get_facebook_posts function to also return comments of each post without increasing the number of API calls, we can make use of the {post_id}/comments edge in the Facebook Graph API.

Here's the modified get_facebook_posts function:
```python
def get_facebook_posts(group_id, access_token, limit=100):
    """
    Makes a GET request to pull posts from a Facebook group using the Facebook API.
    Outputs the results into a DataFrame so that each post is a row along with its time stamp, message,
    permalink, comments count, reactions, and post ID. The function also returns a DataFrame with each post's comments,
    comment message, comment time stamp, and comment author.
    Also outputs the response.json().

    Parameters:
        group_id (str): The ID of the Facebook group to pull posts from.
        access_token (str): An access token with the appropriate permissions to access the group's content.
        limit (int): The number of posts to retrieve (default is 100).

    Returns:
        tuple(pandas.DataFrame, pandas.DataFrame): A tuple of two DataFrames containing the posts and their associated data.
    """
    # Get posts data
    url = f"https://graph.facebook.com/{group_id}/feed"
    post_params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(0).summary(true),reactions.limit(0).summary(true).type(LIKE).as(like),reactions.limit(0).summary(true).type(LOVE).as(love),reactions.limit(0).summary(true).type(WOW).as(wow),reactions.limit(0).summary(true).type(HAHA).as(haha),reactions.limit(0).summary(true).type(SAD).as(sad),reactions.limit(0).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    post_response = requests.get(url, params=post_params)
    post_status_code = post_response.status_code
    post_data = post_response.json()

    # Get comments data
    comments_params = {
        "access_token": access_token,
        "fields": "message,from,created_time",
        "filter": "stream"
    }
    comments_df = pd.DataFrame()
    for post in post_data["data"]:
        post_id = post["id"]
        comments_url = f"https://graph.facebook.com/{post_id}/comments"
        comments_response = requests.get(comments_url, params=comments_params)
        comments_status_code = comments_response.status_code
        comments_data = comments_response.json()
        comments = []
        for comment in comments_data["data"]:
            message = comment.get("message")
            author = comment.get("from", {}).get("name")
            created_time = comment.get("created_time")
            comments.append({
                "post_id": post_id,
                "message": message,
                "author": author,
                "created_time": created_time
            })
        comments_df = pd.concat([comments_df, pd.DataFrame(comments)])
    comments_df.reset_index(drop=True, inplace=True)

    # Get posts DataFrame
    posts = []
    for post in post_data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post

```

## Prompt 2
How do I call the function?

### Response

```python
import requests
import pandas as pd

def get_facebook_posts(group_id, access_token, limit=100):
    url = f"https://graph.facebook.com/{group_id}/feed"
    params = {
        "access_token": access_token,
        "fields": "created_time,message,permalink_url,comments.limit(0).summary(true),reactions.limit(0).summary(true).type(LIKE).as(like),reactions.limit(0).summary(true).type(LOVE).as(love),reactions.limit(0).summary(true).type(WOW).as(wow),reactions.limit(0).summary(true).type(HAHA).as(haha),reactions.limit(0).summary(true).type(SAD).as(sad),reactions.limit(0).summary(true).type(ANGRY).as(angry)",
        "limit": limit
    }
    response = requests.get(url, params=params)
    status_code = response.status_code
    data = response.json()
    posts = []
    comments = []
    for post in data["data"]:
        created_time = post.get("created_time")
        message = post.get("message")
        permalink_url = post.get("permalink_url")
        comments_count = post.get("comments", {}).get("summary", {}).get("total_count", 0)
        reactions = post.get("like", {}).get("summary", {}).get("total_count", 0), post.get("love", {}).get("summary", {}).get("total_count", 0), post.get("wow", {}).get("summary", {}).get("total_count", 0), post.get("haha", {}).get("summary", {}).get("total_count", 0), post.get("sad", {}).get("summary", {}).get("total_count", 0), post.get("angry", {}).get("summary", {}).get("total_count", 0)
        post_id = post.get("id")
        posts.append({
            "post_id": post_id,
            "created_time": created_time,
            "message": message,
            "comments_count": comments_count,
            "like_count": reactions[0],
            "love_count": reactions[1],
            "wow_count": reactions[2],
            "haha_count": reactions[3],
            "sad_count": reactions[4],
            "angry_count": reactions[5],
            "permalink_url": permalink_url,
        })
        # check if there are comments for the post
        if comments_count > 0:
            comments_url = f"https://graph.facebook.com/{post_id}/comments"
            comments_params = {
                "access_token": access_token,
                "fields": "created_time,from,message",
                "limit": comments_count
            }
            comments_response = requests.get(comments_url, params=comments_params)
            comments_data = comments_response.json()
            for comment in comments_data["data"]:
                comment_created_time = comment.get("created_time")
                comment_message = comment.get("message")
                comment_author = comment.get("from", {}).get("name")
                comments.append({
                    "post_id": post_id,
                    "comment_created_time": comment_created_time,
                    "comment_message": comment_message,
                    "comment_author": comment_author
                })
    # Create DataFrames
    posts_df = pd.DataFrame(posts)
    comments_df = pd.DataFrame(comments)
    # Reorder columns
    posts_df = posts_df[["post_id", "created_time", "message", "comments_count", "like_count", "love_count", "wow

```

### My thoughts: Code is incomplete.

# *End of Page*